In [48]:
import numpy as np
from PIL import Image
import brotli
import struct
from io import BytesIO
from collections import Counter
import heapq

### --- Util Functions --- ###

def calculate_image_variance(img_arr):
    return np.var(img_arr)

def delta_encode_rgb(img_array):
    delta = np.zeros_like(img_array)
    delta[0] = img_array[0]
    delta[1:] = (img_array[1:].astype(np.int16) - img_array[:-1].astype(np.int16)) % 256
    return delta.astype(np.uint8)

def bitpack_rgb(img_array):
    packed = bytearray()
    for r, g, b in img_array:
        packed.append(((r >> 4) << 4) | (g >> 4))  # R and G
        packed.append((b >> 4) << 4)               # B and 4-bit padding (0)
    return bytes(packed)


### --- Huffman Compression --- ###
class Node:
    def __init__(self, char=None, freq=None):
        self.char = char
        self.freq = freq
        self.left = None
        self.right = None
    def __lt__(self, other):
        return self.freq < other.freq

def build_huffman_tree(data):
    frequency = Counter(data)
    heap = [Node(char=k, freq=v) for k, v in frequency.items()]
    heapq.heapify(heap)
    while len(heap) > 1:
        n1 = heapq.heappop(heap)
        n2 = heapq.heappop(heap)
        merged = Node(freq=n1.freq + n2.freq)
        merged.left = n1
        merged.right = n2
        heapq.heappush(heap, merged)
    return heap[0] if heap else None

def build_huffman_dict(node, prefix="", codebook=None):
    if codebook is None:
        codebook = {}
    if node:
        if node.char is not None:
            codebook[node.char] = prefix
        build_huffman_dict(node.left, prefix + "0", codebook)
        build_huffman_dict(node.right, prefix + "1", codebook)
    return codebook

def compress_huffman(data):
    tree = build_huffman_tree(data)
    codebook = build_huffman_dict(tree)
    encoded_bits = ''.join(codebook[byte] for byte in data)
    padding = 8 - len(encoded_bits) % 8
    encoded_bits += '0' * padding
    b = bytearray()
    for i in range(0, len(encoded_bits), 8):
        b.append(int(encoded_bits[i:i+8], 2))
    return bytes(b)

### --- Compression Strategies --- ###

def compress_delta_huffman_with_codebook(img_arr):
    delta = delta_encode_rgb(img_arr)
    flat = delta.flatten().tobytes()
    tree = build_huffman_tree(flat)
    codebook = build_huffman_dict(tree)

    # Encode bitstream
    encoded_bits = ''.join(codebook[byte] for byte in flat)
    padding = (8 - len(encoded_bits) % 8) % 8
    encoded_bits += '0' * padding
    compressed = bytearray(int(encoded_bits[i:i+8], 2) for i in range(0, len(encoded_bits), 8))

    # Serialize codebook
    cb = bytearray()
    cb.append(len(codebook))  # # of entries
    for byte, code in codebook.items():
        cb.append(byte)
        cb.append(len(code))  # code length in bits
        cb.extend(int(code, 2).to_bytes((len(code) + 7) // 8, 'big'))

    return bytes(cb) + bytes([padding]) + bytes(compressed)



def compress_bitpack_brotli(img_arr):
    packed = bitpack_rgb(img_arr)
    return brotli.compress(packed)

def compress_delta_bitpack_brotli(img_arr):
    delta = delta_encode_rgb(img_arr)
    packed = bitpack_rgb(delta)
    return brotli.compress(packed)

### --- Final Smart Encoder --- ###

def encode_lix_rgb(image_path, output_path):
    img = Image.open(image_path).convert("RGB")
    img_arr = np.array(img).reshape(-1, 3)
    height, width = img.height, img.width

    variance = calculate_image_variance(img_arr)
    
    # Compression selector based on image complexity
    if variance < 300:
        method = 1  # Delta + Huffman
        compressed = compress_delta_huffman_with_codebook(img_arr)
    elif variance < 800:
        method = 2  # Bitpack + Brotli
        compressed = compress_bitpack_brotli(img_arr)
    else:
        method = 3  # Delta + Bitpack + Brotli
        compressed = compress_delta_bitpack_brotli(img_arr)

    # Header Format: [magic (4 bytes)] [width (2)] [height (2)] [type (1=RGB)] [method (1)]
    header = b'LIXF' + struct.pack(">HHBB", width, height, 0x03, method)

    with open(output_path, 'wb') as f:
        f.write(header + compressed)

    print(f"✅ Saved .lix RGB file: {output_path}, size: {len(header + compressed)} bytes, method: {method}")

encode_lix_rgb("exp6.jpg", "image_rgb_exp4.lix")



✅ Saved .lix RGB file: image_rgb_exp4.lix, size: 9382 bytes, method: 2


In [49]:
import struct
import brotli
import numpy as np
from PIL import Image
from io import BytesIO
from collections import Counter
import heapq

### --- Huffman Decompression --- ###

class Node:
    def __init__(self, char=None, freq=None):
        self.char = char
        self.freq = freq
        self.left = None
        self.right = None
    def __lt__(self, other):
        return self.freq < other.freq

def build_huffman_tree_from_dict(codebook):
    root = Node()
    for byte, code in codebook.items():
        current = root
        for bit in code:
            if bit == '0':
                if not current.left:
                    current.left = Node()
                current = current.left
            else:
                if not current.right:
                    current.right = Node()
                current = current.right
        current.char = byte
    return root

def decompress_huffman(data, codebook):
    bitstring = ''.join(f"{byte:08b}" for byte in data)
    root = build_huffman_tree_from_dict(codebook)
    result = bytearray()
    node = root
    for bit in bitstring:
        node = node.left if bit == '0' else node.right
        if node.char is not None:
            result.append(node.char)
            node = root
    return bytes(result)

### --- Util Functions --- ###

def delta_decode_rgb(delta_array):
    decoded = np.zeros_like(delta_array, dtype=np.uint8)
    decoded[0] = delta_array[0]
    for i in range(1, len(delta_array)):
        decoded[i] = (decoded[i-1].astype(np.int16) + delta_array[i]) % 256
    return decoded.astype(np.uint8)

def bitunpack_rgb(packed_bytes):
    pixels = []
    for i in range(0, len(packed_bytes), 2):
        byte1 = packed_bytes[i]
        byte2 = packed_bytes[i + 1]
        r = (byte1 >> 4) << 4
        g = (byte1 & 0x0F) << 4
        b = (byte2 >> 4) << 4
        pixels.append((r, g, b))
    return np.array(pixels, dtype=np.uint8)


### --- Dummy Codebook Generator (for testing) --- ###
# In real implementation, we would store or regenerate the same Huffman codebook used in compression.
def get_dummy_huffman_codebook(data):
    freq = Counter(data)
    heap = [Node(k, v) for k, v in freq.items()]
    heapq.heapify(heap)
    while len(heap) > 1:
        n1 = heapq.heappop(heap)
        n2 = heapq.heappop(heap)
        merged = Node(freq=n1.freq + n2.freq)
        merged.left, merged.right = n1, n2
        heapq.heappush(heap, merged)
    return build_huffman_dict(heap[0])

### --- Final Smart Decoder --- ###

def decode_lix_rgb(file_path, output_path):
    with open(file_path, 'rb') as f:
        content = f.read()

    header = content[:10]
    magic, width, height, img_type, method = struct.unpack(">4sHHBB", header)
    assert magic == b'LIXF' and img_type == 0x03, "Not a valid RGB .lix file"
    data = content[10:]

    # Decompression based on method
    if method == 1:
        # Deserialize codebook
        ptr = 0
        num_entries = data[ptr]
        ptr += 1
        codebook = {}
        for _ in range(num_entries):
            byte = data[ptr]
            ptr += 1
            length = data[ptr]
            ptr += 1
            num_bytes = (length + 7) // 8
            code_bytes = data[ptr:ptr + num_bytes]
            ptr += num_bytes
            code = bin(int.from_bytes(code_bytes, 'big'))[2:].zfill(length)
            codebook[byte] = code

        padding = data[ptr]
        ptr += 1
        bitstream_data = data[ptr:]
        
        # Decode
        bitstring = ''.join(f"{b:08b}" for b in bitstream_data)
        bitstring = bitstring[:-padding] if padding else bitstring
        root = build_huffman_tree_from_dict(codebook)
        result = bytearray()
        node = root
        for bit in bitstring:
            node = node.left if bit == '0' else node.right
            if node.char is not None:
                result.append(node.char)
                node = root

        delta_arr = np.frombuffer(result, dtype=np.uint8).reshape(-1, 3)
        img_arr = delta_decode_rgb(delta_arr)

    elif method == 2:
        # Bitpack + Brotli
        packed = brotli.decompress(data)
        img_arr = bitunpack_rgb(packed)
    elif method == 3:
        # Delta + Bitpack + Brotli
        packed = brotli.decompress(data)
        delta_arr = bitunpack_rgb(packed)
        img_arr = delta_decode_rgb(delta_arr)
    else:
        raise ValueError("Unsupported compression method")

    # Reshape and save
    img_arr = img_arr.reshape((height, width, 3))
    img = Image.fromarray(img_arr, 'RGB')
    img.save(output_path)
    print(f"✅ Decoded and saved RGB image to {output_path}")

decode_lix_rgb("image_rgb_exp4.lix", "decoded_rgb_exp4.png")


✅ Decoded and saved RGB image to decoded_rgb_exp4.png


In [ ]:
import numpy as np
from PIL import Image
import brotli
import struct
from io import BytesIO
from collections import Counter
import heapq

### --- Util Functions --- ###

def calculate_image_variance(img_arr):
    return np.var(img_arr)

def delta_encode_rgb(img_array):
    delta = np.zeros_like(img_array)
    delta[0] = img_array[0]
    delta[1:] = (img_array[1:].astype(np.int16) - img_array[:-1].astype(np.int16)) % 256
    return delta.astype(np.uint8)

def delta_decode_rgb(delta_array):
    decoded = np.zeros_like(delta_array, dtype=np.uint8)
    decoded[0] = delta_array[0]
    for i in range(1, len(delta_array)):
        decoded[i] = (decoded[i-1].astype(np.int16) + delta_array[i]) % 256
    return decoded.astype(np.uint8)

def bitpack_rgb(img_array):
    packed = bytearray()
    for r, g, b in img_array:
        packed.append(((r >> 4) << 4) | (g >> 4))  # R and G
        packed.append((b >> 4) << 4)               # B and 4-bit padding (0)
    return bytes(packed)

def bitunpack_rgb(packed_bytes):
    pixels = []
    for i in range(0, len(packed_bytes), 2):
        byte1 = packed_bytes[i]
        byte2 = packed_bytes[i + 1]
        r = (byte1 >> 4) << 4
        g = (byte1 & 0x0F) << 4
        b = (byte2 >> 4) << 4
        pixels.append((r, g, b))
    return np.array(pixels, dtype=np.uint8)


### --- Huffman Compression --- ###
class Node:
    def __init__(self, char=None, freq=None):
        self.char = char
        self.freq = freq
        self.left = None
        self.right = None
    def __lt__(self, other):
        return self.freq < other.freq

def build_huffman_tree(data):
    frequency = Counter(data)
    heap = [Node(char=k, freq=v) for k, v in frequency.items()]
    heapq.heapify(heap)
    while len(heap) > 1:
        n1 = heapq.heappop(heap)
        n2 = heapq.heappop(heap)
        merged = Node(freq=n1.freq + n2.freq)
        merged.left = n1
        merged.right = n2
        heapq.heappush(heap, merged)
    return heap[0] if heap else None

def build_huffman_dict(node, prefix="", codebook=None):
    if codebook is None:
        codebook = {}
    if node:
        if node.char is not None:
            codebook[node.char] = prefix
        build_huffman_dict(node.left, prefix + "0", codebook)
        build_huffman_dict(node.right, prefix + "1", codebook)
    return codebook

def compress_huffman(data):
    tree = build_huffman_tree(data)
    codebook = build_huffman_dict(tree)
    encoded_bits = ''.join(codebook[byte] for byte in data)
    padding = 8 - len(encoded_bits) % 8
    encoded_bits += '0' * padding
    b = bytearray()
    for i in range(0, len(encoded_bits), 8):
        b.append(int(encoded_bits[i:i+8], 2))
    return bytes(b)


def build_huffman_tree_from_dict(codebook):
    root = Node()
    for byte, code in codebook.items():
        current = root
        for bit in code:
            if bit == '0':
                if not current.left:
                    current.left = Node()
                current = current.left
            else:
                if not current.right:
                    current.right = Node()
                current = current.right
        current.char = byte
    return root

def decompress_huffman(data, codebook, padding):
    bitstring = ''.join(f"{byte:08b}" for byte in data)
    bitstring = bitstring[:-padding] if padding else bitstring
    root = build_huffman_tree_from_dict(codebook)
    result = bytearray()
    node = root
    for bit in bitstring:
        node = node.left if bit == '0' else node.right
        if node.char is not None:
            result.append(node.char)
            node = root
    return bytes(result)



### --- Compression Strategies --- ###

def compress_delta_huffman_with_codebook(img_arr):
    delta = delta_encode_rgb(img_arr)
    flat = delta.flatten().tobytes()
    tree = build_huffman_tree(flat)
    codebook = build_huffman_dict(tree)

    # Encode bitstream
    encoded_bits = ''.join(codebook[byte] for byte in flat)
    padding = (8 - len(encoded_bits) % 8) % 8
    encoded_bits += '0' * padding
    compressed = bytearray(int(encoded_bits[i:i+8], 2) for i in range(0, len(encoded_bits), 8))

    # Serialize codebook
    cb = bytearray()
    cb.append(len(codebook))  # # of entries
    for byte, code in codebook.items():
        cb.append(byte)
        cb.append(len(code))  # code length in bits
        cb.extend(int(code, 2).to_bytes((len(code) + 7) // 8, 'big'))

    return bytes(cb) + bytes([padding]) + bytes(compressed)


def compress_delta_huffman(img_arr):
    delta = delta_encode_rgb(img_arr)
    flat = delta.flatten().tobytes()
    compressed = compress_huffman(flat)
    return compressed

def compress_bitpack_brotli(img_arr):
    packed = bitpack_rgb(img_arr)
    return brotli.compress(packed)

def compress_delta_bitpack_brotli(img_arr):
    delta = delta_encode_rgb(img_arr)
    packed = bitpack_rgb(delta)
    return brotli.compress(packed)

def compress_brotli(img_arr):
    flat = img_arr.flatten().tobytes()
    return brotli.compress(flat)


### --- Final Smart Encoder --- ###

def encode_lix_rgb(image_path, output_path):
    img = Image.open(image_path).convert("RGB")
    img_arr = np.array(img)
    height, width = img.height, img.width
    img_arr_flat = img_arr.reshape(-1, 3)

    variance = calculate_image_variance(img_arr_flat)
    
    # Compression selector based on image complexity
    if variance < 300:
        method = 1  # Delta + Huffman
        compressed = compress_delta_huffman_with_codebook(img_arr_flat)
    elif variance < 800:
        method = 2  # Bitpack + Brotli
        compressed = compress_bitpack_brotli(img_arr_flat)
    elif variance < 1500:
        method = 3  # Delta + Bitpack + Brotli
        compressed = compress_delta_bitpack_brotli(img_arr_flat)
    else:
        method = 4  # Brotli
        compressed = compress_brotli(img_arr_flat)

    # Header Format: [magic (4 bytes)] [width (2)] [height (2)] [type (1=RGB)] [method (1)]
    header = b'LIXF' + struct.pack(">HHBB", width, height, 0x03, method)

    with open(output_path, 'wb') as f:
        f.write(header + compressed)

    print(f"✅ Saved .lix RGB file: {output_path}, size: {len(header + compressed)} bytes, method: {method}")


### --- DECODER --- ###
def decode_lix_rgb(file_path, output_path):
    with open(file_path, 'rb') as f:
        content = f.read()

    header = content[:10]
    magic, width, height, img_type, method = struct.unpack(">4sHHBB", header)
    assert magic == b'LIXF' and img_type == 0x03, "Not a valid RGB .lix file"
    data = content[10:]

    # Decompression based on method
    if method == 1:
        # Deserialize codebook
        ptr = 0
        num_entries = data[ptr]
        ptr += 1
        codebook = {}
        for _ in range(num_entries):
            byte = data[ptr]
            ptr += 1
            length = data[ptr]
            ptr += 1
            num_bytes = (length + 7) // 8
            code_bytes = data[ptr:ptr + num_bytes]
            ptr += num_bytes
            code = bin(int.from_bytes(code_bytes, 'big'))[2:].zfill(length)
            codebook[byte] = code

        padding = data[ptr]
        ptr += 1
        bitstream_data = data[ptr:]
        
        # Decode
        flat = decompress_huffman(bitstream_data, codebook, padding)
        delta_arr = np.frombuffer(flat, dtype=np.uint8).reshape(-1, 3)
        img_arr = delta_decode_rgb(delta_arr)


    elif method == 2:
        # Bitpack + Brotli
        packed = brotli.decompress(data)
        img_arr = bitunpack_rgb(packed)
    elif method == 3:
        # Delta + Bitpack + Brotli
        packed = brotli.decompress(data)
        delta_arr = bitunpack_rgb(packed)
        img_arr = delta_decode_rgb(delta_arr)
    elif method == 4:
        flat = brotli.decompress(data)
        img_arr = np.frombuffer(flat, dtype=np.uint8).reshape(height, width, 3)
    else:
        raise ValueError("Unsupported compression method")

    # Reshape and save
    if method != 4:
        img_arr = img_arr.reshape((height, width, 3))
    img = Image.fromarray(img_arr, 'RGB')
    img.save(output_path)
    print(f"✅ Decoded and saved RGB image to {output_path}")

# Example Usage
image_path = "exp.jpg"  # Replace with the actual path to your image
encoded_path = "image_rgb_exp4.lix"
decoded_path = "decoded_rgb_exp4.png"

encode_lix_rgb(image_path, encoded_path)
decode_lix_rgb(encoded_path, decoded_path)

✅ Saved .lix RGB file: image_rgb_exp4.lix, size: 20836 bytes, method: 4
✅ Decoded and saved RGB image to decoded_rgb_exp4.png


In [ ]:
import numpy as np
from PIL import Image
import brotli
import struct
from io import BytesIO
from collections import Counter
import heapq


### --- Utility Functions --- ###
def calculate_image_variance(img_arr):
    return np.var(img_arr)

def delta_encode_rgb(img_array):
    delta = np.zeros_like(img_array)
    delta[0] = img_array[0]
    delta[1:] = (img_array[1:].astype(np.int16) - img_array[:-1].astype(np.int16)) % 256
    return delta.astype(np.uint8)

def delta_decode_rgb(delta_array):
    decoded = np.zeros_like(delta_array, dtype=np.uint8)
    decoded[0] = delta_array[0]
    for i in range(1, len(delta_array)):
        decoded[i] = (decoded[i-1].astype(np.int16) + delta_array[i]) % 256
    return decoded

def bitpack_rgb(img_array):
    packed = bytearray()
    for r, g, b in img_array:
        packed.append(((r >> 4) << 4) | (g >> 4))
        packed.append((b >> 4) << 4)
    return bytes(packed)

def bitunpack_rgb(packed_bytes):
    pixels = []
    for i in range(0, len(packed_bytes), 2):
        byte1, byte2 = packed_bytes[i], packed_bytes[i + 1]
        r = (byte1 >> 4) << 4
        g = (byte1 & 0x0F) << 4
        b = (byte2 >> 4) << 4
        pixels.append((r, g, b))
    return np.array(pixels, dtype=np.uint8)


### --- Huffman --- ###
class Node:
    def __init__(self, char=None, freq=None):
        self.char = char
        self.freq = freq
        self.left = self.right = None
    def __lt__(self, other): return self.freq < other.freq

def build_huffman_tree(data):
    freq = Counter(data)
    heap = [Node(k, v) for k, v in freq.items()]
    heapq.heapify(heap)
    while len(heap) > 1:
        left, right = heapq.heappop(heap), heapq.heappop(heap)
        parent = Node(freq=left.freq + right.freq)
        parent.left, parent.right = left, right
        heapq.heappush(heap, parent)
    return heap[0]

def build_huffman_dict(node, prefix='', codebook=None):
    if codebook is None: codebook = {}
    if node.char is not None:
        codebook[node.char] = prefix
    else:
        build_huffman_dict(node.left, prefix + '0', codebook)
        build_huffman_dict(node.right, prefix + '1', codebook)
    return codebook

def compress_huffman_with_codebook(data):
    tree = build_huffman_tree(data)
    codebook = build_huffman_dict(tree)
    encoded_bits = ''.join(codebook[byte] for byte in data)
    padding = (8 - len(encoded_bits) % 8) % 8
    encoded_bits += '0' * padding
    compressed = bytearray(int(encoded_bits[i:i+8], 2) for i in range(0, len(encoded_bits), 8))

    # Serialize codebook
    cb = bytearray()
    cb.append(len(codebook))
    for byte, code in codebook.items():
        cb.append(byte)
        cb.append(len(code))
        cb.extend(int(code, 2).to_bytes((len(code) + 7) // 8, 'big'))
    return bytes(cb) + bytes([padding]) + bytes(compressed)

def build_huffman_tree_from_dict(codebook):
    root = Node()
    for byte, code in codebook.items():
        node = root
        for bit in code:
            if bit == '0':
                node.left = node.left or Node()
                node = node.left
            else:
                node.right = node.right or Node()
                node = node.right
        node.char = byte
    return root

def decompress_huffman(data, codebook, padding):
    bitstring = ''.join(f"{byte:08b}" for byte in data)
    bitstring = bitstring[:-padding] if padding else bitstring
    root = build_huffman_tree_from_dict(codebook)
    node, output = root, bytearray()
    for bit in bitstring:
        node = node.left if bit == '0' else node.right
        if node.char is not None:
            output.append(node.char)
            node = root
    return bytes(output)


### --- Compression Strategies --- ###
def compress_delta_huffman_with_codebook(img_arr):
    delta = delta_encode_rgb(img_arr)
    return compress_huffman_with_codebook(delta.flatten())

def compress_bitpack_brotli(img_arr):
    return brotli.compress(bitpack_rgb(img_arr))

def compress_delta_bitpack_brotli(img_arr):
    return brotli.compress(bitpack_rgb(delta_encode_rgb(img_arr)))

def compress_brotli(img_arr):
    return brotli.compress(img_arr.flatten().tobytes())


### --- Encoder --- ###


def encode_lix_rgb(image_path, output_path):
    img = Image.open(image_path).convert("RGB")
    img_arr = np.array(img).reshape(-1, 3)
    height, width = img.height, img.width
    variance = calculate_image_variance(img_arr)

    if variance < 300:
        method = 1; compressed = compress_delta_huffman_with_codebook(img_arr)
    elif variance < 800:
        method = 2; compressed = compress_bitpack_brotli(img_arr)
    elif variance < 1500:
        method = 3; compressed = compress_delta_bitpack_brotli(img_arr)
    else:
        method = 4; compressed = compress_brotli(img_arr)

    header = b'LIXF' + struct.pack(">HHBB", width, height, 0x03, method)
    with open(output_path, 'wb') as f:
        f.write(header + compressed)

    print(f"✅ Saved .lix RGB file: {output_path}, size: {len(header + compressed)} bytes, method: {method}")




### --- Decoder --- ###
def decode_lix_rgb(file_path, output_path, quality=85):  # Added quality parameter
    with open(file_path, 'rb') as f:
        content = f.read()

    header = content[:10]
    magic, width, height, img_type, method = struct.unpack(">4sHHBB", header)
    assert magic == b'LIXF' and img_type == 0x03, "Invalid .lix RGB file"
    data = content[10:]

    if method == 1:
        ptr = 0
        entries = data[ptr]; ptr += 1
        codebook = {}
        for _ in range(entries):
            byte = data[ptr]; ptr += 1
            length = data[ptr]; ptr += 1
            code_len = (length + 7) // 8
            code = bin(int.from_bytes(data[ptr:ptr + code_len], 'big'))[2:].zfill(length)
            codebook[byte] = code
            ptr += code_len
        padding = data[ptr]; ptr += 1
        flat = decompress_huffman(data[ptr:], codebook, padding)
        delta_arr = np.frombuffer(flat, dtype=np.uint8).reshape(-1, 3)
        img_arr = delta_decode_rgb(delta_arr)

    elif method == 2:
        img_arr = bitunpack_rgb(brotli.decompress(data))
    elif method == 3:
        delta_arr = bitunpack_rgb(brotli.decompress(data))
        img_arr = delta_decode_rgb(delta_arr)
    elif method == 4:
        img_arr = np.frombuffer(brotli.decompress(data), dtype=np.uint8).reshape(height, width, 3)
    else:
        raise ValueError("Unsupported method")

    if method != 4:
        img_arr = img_arr.reshape((height, width, 3))

    img = Image.fromarray(img_arr, 'RGB')
    img.save(output_path, "JPEG", quality=quality)  # Save as JPG with quality
    print(f"✅ Decoded and saved to {output_path} as JPEG with quality {quality}")



if __name__ == "__main__":
    image_path = "exp4.jpg" # replace with a path to your image
    encoded_path = "image_rgb_exp4.lix"
    decoded_path = "decoded_rgb_exp4.jpg"  # Save as JPG

    encode_lix_rgb(image_path, encoded_path)
    decode_lix_rgb(encoded_path, decoded_path)  # Pass JPG to decoder

✅ Saved .lix RGB file: image_rgb_exp4.lix, size: 4278 bytes, method: 4
✅ Decoded and saved to decoded_rgb_exp4.jpg as JPEG with quality 85
